In [1]:
#importing all the required libraries and modules
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
from pandas.tseries.holiday import USFederalHolidayCalendar
import scipy.stats
import matplotlib.dates as mdates
from google.cloud import bigquery

In [2]:
#connecting notebook to gcloud bigquery to query data.
cred_file = r"C:\Users\V0V01S7\AppData\Roaming\gcloud\legacy_credentials\venkatasaiyashaswi.v@walmart.com\adc.json"
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]=cred_file
def query_bq(QUERY):
    client = bigquery.Client(project='wmt-edw-sandbox')
    query_job = client.query(QUERY) # API request
    rows = query_job.result() # Waits for query to finish
    df = rows.to_dataframe()
    return df

In [3]:
code = """select A.DRVR_USER_ID, 
B.ACTV_START_TS, B.ACTL_ACTV_END_TS, B.ACTV_DUR_MIN_QTY, B.ACTV_DT, 
from `wmt-edw-prod.WW_GEC_VM.DRVR` A inner join `wmt-edw-prod.WW_GEC_VM.DRVR_APPLN_ACTV` B on A.DRVR_USER_ID = B.DRVR_USER_ID
where A.MARKET_NM in ('Walmart San Jose North','Walmart San Jose South','Walmart Orlando','Walmart Virginia Beach', 'Walmart Tulsa','Walmart South Minneapolis') AND B.ACTV_NM = "ONLINE"
"""
df = query_bq(code)
df["ACTV_START_TS"] = pd.to_datetime(df["ACTV_START_TS"])
df["ACTL_ACTV_END_TS"] = pd.to_datetime(df["ACTL_ACTV_END_TS"])
df.head()

,DRVR_USER_ID,ACTV_START_TS,ACTL_ACTV_END_TS,ACTV_DUR_MIN_QTY,ACTV_DT
0,andresrmaestre18@gmail.com,2021-07-27 04:04:09.164,2021-07-27 06:21:20.992,137.180000000,2021-07-27
1,andresrmaestre18@gmail.com,2021-07-27 06:21:20.992,2021-07-27 09:30:28.519,189.130000000,2021-07-27
2,andresrmaestre18@gmail.com,2021-07-27 09:30:28.519,2021-07-27 11:58:18.767,147.830000000,2021-07-27
3,andresrmaestre18@gmail.com,2021-07-27 11:58:18.767,2021-07-27 16:00:00.000,241.700000000,2021-07-27
4,ffarsi@cox.net,2021-07-27 11:09:02.752,2021-07-27 15:00:00.000,230.970000000,2021-07-27


In [4]:
def final_ft(df):
    df['dayhour'] = df["ACTV_START_TS"].apply(lambda x: x.replace(minute=0, second=0, microsecond=0))
    time_line = pd.date_range(df["dayhour"].min(),df["dayhour"].max(),freq='H')
    time_line = time_line.to_series()
    ts = pd.DataFrame(time_line,columns = ["dayhour"])
    ts = ts.set_index("dayhour")
    ts = pd.merge(ts,df,how="left",on=["dayhour"])
    ts["ACTV_DUR_MIN_QTY"] = ts["ACTV_DUR_MIN_QTY"].fillna(0)
    ts = ts.drop(["DRVR_USER_ID","ACTV_DT"],axis=1)
    ts = ts.set_index(["dayhour"],drop=True)
    ts = ts.drop_duplicates()
    ts = ts.dropna()
    ts = ts.reset_index(drop=False)
    ts['ACTV_START_TS'] = ts["ACTV_START_TS"].apply(lambda x: x.replace(second=0, microsecond=0))
    ts['ACTL_ACTV_END_TS'] = ts["ACTL_ACTV_END_TS"].apply(lambda x: x.replace(second=0, microsecond=0))
    def create_final_data(df):
        df['minutes'] = (df['ACTL_ACTV_END_TS'] - df['ACTV_START_TS']).dt.total_seconds()/60
        df = df[df['minutes']>0].reset_index(drop=True)
        iv_idx = pd.IntervalIndex.from_arrays(df['ACTV_START_TS'],
                                              df['ACTL_ACTV_END_TS']-pd.Timedelta(minutes=1),
                                              closed='both')
        new_idx = pd.date_range(df['ACTV_START_TS'].min(), df['ACTL_ACTV_END_TS'].max(), freq='min')
        result = df['minutes'].set_axis(iv_idx).reindex(new_idx)
        result= result.fillna(0).astype(int).astype(bool).resample('H').sum()
        result.index.name = 'start_hour'
        return result
    re = create_final_data(ts)
    re = pd.DataFrame(re)
    def create_features(df, label=None):
        df = df.copy()
        df['date'] = df.index
        df['hour'] = df['date'].dt.hour
        df['dayofweek'] = df['date'].dt.dayofweek
        df['quarter'] = df['date'].dt.quarter
        df['month'] = df['date'].dt.month
        df['year'] = df['date'].dt.year
        df['dayofyear'] = df['date'].dt.dayofyear
        df['dayofmonth'] = df['date'].dt.day
        df['weekofyear'] = df['date'].dt.isocalendar().week

        X = df[['hour','dayofweek','quarter','month','year',
               'dayofyear','dayofmonth','weekofyear']]
        if label:
            y = df[label]
            return X, y
        return X
    X, y = create_features(re, label='minutes')
    ft = pd.concat([X, y], axis=1)    
    return ft

In [5]:
df.drop((df.loc[df["DRVR_USER_ID"].str.contains("walmart|faisal",case=False)]).index, inplace = True)

In [6]:
df = df[df["ACTV_DUR_MIN_QTY"] > 0 ]
df = df.drop_duplicates()

In [12]:
df["ACTV_DT"] = pd.to_datetime(df["ACTV_DT"])
df = df[df['ACTV_DT'] >= '2022-05-01']

In [18]:
df = df.set_index('DRVR_USER_ID',drop = True)

In [21]:
df = df.reset_index()

In [25]:
def slot_calc(hour):
    if hour in range(6,11):
        return "morning"
    elif hour in range(11,17):
        return "afternoon"
    elif hour in range(17,23):
        return "evening"

In [26]:
dfs = {}
for ID in df['DRVR_USER_ID'].value_counts().index.tolist():
    dfs[ID] = df.loc[df['DRVR_USER_ID'] == ID]

In [27]:
dfs_set = {}
for i in dfs.keys():
    dfs_set[i] = pd.DataFrame(final_ft(dfs[i]))
    dfs_set[i]["ID"] = i

C:\Users\V0V01S7\AppData\Local\Temp\ipykernel_13184\3165893152.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['dayhour'] = df["ACTV_START_TS"].apply(lambda x: x.replace(minute=0, second=0, microsecond=0))
C:\Users\V0V01S7\AppData\Local\Temp\ipykernel_13184\3165893152.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['dayhour'] = df["ACTV_START_TS"].apply(lambda x: x.replace(minute=0, second=0, microsecond=0))
C:\Users\V0V01S7\AppData\Local\Temp\ipykernel_13184\3165893152.py:2: SettingWithCopyWa

C:\Users\V0V01S7\AppData\Local\Temp\ipykernel_13184\3165893152.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['dayhour'] = df["ACTV_START_TS"].apply(lambda x: x.replace(minute=0, second=0, microsecond=0))
C:\Users\V0V01S7\AppData\Local\Temp\ipykernel_13184\3165893152.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['dayhour'] = df["ACTV_START_TS"].apply(lambda x: x.replace(minute=0, second=0, microsecond=0))
C:\Users\V0V01S7\AppData\Local\Temp\ipykernel_13184\3165893152.py:2: SettingWithCopyWa

C:\Users\V0V01S7\AppData\Local\Temp\ipykernel_13184\3165893152.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['dayhour'] = df["ACTV_START_TS"].apply(lambda x: x.replace(minute=0, second=0, microsecond=0))
C:\Users\V0V01S7\AppData\Local\Temp\ipykernel_13184\3165893152.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['dayhour'] = df["ACTV_START_TS"].apply(lambda x: x.replace(minute=0, second=0, microsecond=0))
C:\Users\V0V01S7\AppData\Local\Temp\ipykernel_13184\3165893152.py:2: SettingWithCopyWa

C:\Users\V0V01S7\AppData\Local\Temp\ipykernel_13184\3165893152.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['dayhour'] = df["ACTV_START_TS"].apply(lambda x: x.replace(minute=0, second=0, microsecond=0))
C:\Users\V0V01S7\AppData\Local\Temp\ipykernel_13184\3165893152.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['dayhour'] = df["ACTV_START_TS"].apply(lambda x: x.replace(minute=0, second=0, microsecond=0))
C:\Users\V0V01S7\AppData\Local\Temp\ipykernel_13184\3165893152.py:2: SettingWithCopyWa

C:\Users\V0V01S7\AppData\Local\Temp\ipykernel_13184\3165893152.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['dayhour'] = df["ACTV_START_TS"].apply(lambda x: x.replace(minute=0, second=0, microsecond=0))
C:\Users\V0V01S7\AppData\Local\Temp\ipykernel_13184\3165893152.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['dayhour'] = df["ACTV_START_TS"].apply(lambda x: x.replace(minute=0, second=0, microsecond=0))
C:\Users\V0V01S7\AppData\Local\Temp\ipykernel_13184\3165893152.py:2: SettingWithCopyWa

...

C:\Users\V0V01S7\AppData\Local\Temp\ipykernel_13184\3165893152.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['dayhour'] = df["ACTV_START_TS"].apply(lambda x: x.replace(minute=0, second=0, microsecond=0))
C:\Users\V0V01S7\AppData\Local\Temp\ipykernel_13184\3165893152.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['dayhour'] = df["ACTV_START_TS"].apply(lambda x: x.replace(minute=0, second=0, microsecond=0))
C:\Users\V0V01S7\AppData\Local\Temp\ipykernel_13184\3165893152.py:2: SettingWithCopyWa

C:\Users\V0V01S7\AppData\Local\Temp\ipykernel_13184\3165893152.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['dayhour'] = df["ACTV_START_TS"].apply(lambda x: x.replace(minute=0, second=0, microsecond=0))
C:\Users\V0V01S7\AppData\Local\Temp\ipykernel_13184\3165893152.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['dayhour'] = df["ACTV_START_TS"].apply(lambda x: x.replace(minute=0, second=0, microsecond=0))
C:\Users\V0V01S7\AppData\Local\Temp\ipykernel_13184\3165893152.py:2: SettingWithCopyWa

C:\Users\V0V01S7\AppData\Local\Temp\ipykernel_13184\3165893152.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['dayhour'] = df["ACTV_START_TS"].apply(lambda x: x.replace(minute=0, second=0, microsecond=0))
C:\Users\V0V01S7\AppData\Local\Temp\ipykernel_13184\3165893152.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['dayhour'] = df["ACTV_START_TS"].apply(lambda x: x.replace(minute=0, second=0, microsecond=0))
C:\Users\V0V01S7\AppData\Local\Temp\ipykernel_13184\3165893152.py:2: SettingWithCopyWa

C:\Users\V0V01S7\AppData\Local\Temp\ipykernel_13184\3165893152.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['dayhour'] = df["ACTV_START_TS"].apply(lambda x: x.replace(minute=0, second=0, microsecond=0))
C:\Users\V0V01S7\AppData\Local\Temp\ipykernel_13184\3165893152.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['dayhour'] = df["ACTV_START_TS"].apply(lambda x: x.replace(minute=0, second=0, microsecond=0))
C:\Users\V0V01S7\AppData\Local\Temp\ipykernel_13184\3165893152.py:2: SettingWithCopyWa

ValueError: Neither `start` nor `end` can be NaT

In [123]:
dff = pd.concat([dfs_set[i] for i in dfs_set.keys()], ignore_index=False)

In [124]:
dff

,hour,dayofweek,quarter,month,year,dayofyear,dayofmonth,weekofyear,minutes,ID
start_hour,,,,,,,,,,
2022-05-01 04:00:00,4,6,2,5,2022,121,1,17,14,07familylife@gmail.com
2022-05-01 05:00:00,5,6,2,5,2022,121,1,17,15,07familylife@gmail.com
2022-05-01 06:00:00,6,6,2,5,2022,121,1,17,0,07familylife@gmail.com
2022-05-01 07:00:00,7,6,2,5,2022,121,1,17,0,07familylife@gmail.com
2022-05-01 08:00:00,8,6,2,5,2022,121,1,17,0,07familylife@gmail.com
...,...,...,...,...,...,...,...,...,...,...
2022-07-17 04:00:00,4,6,3,7,2022,198,17,28,0,jdsmith620@outlook.com
2022-07-17 05:00:00,5,6,3,7,2022,198,17,28,0,jdsmith620@outlook.com
2022-07-17 06:00:00,6,6,3,7,2022,198,17,28,0,jdsmith620@outlook.com


In [125]:
dff['market'] = dff['ID'].map(drvr_dict)

In [128]:
virg_dff = dff[dff['market'] == 'Walmart Virginia Beach']

In [130]:
virg_dff.head()

,hour,dayofweek,quarter,month,year,dayofyear,dayofmonth,weekofyear,minutes,ID,market
start_hour,,,,,,,,,,,
2022-05-01 04:00:00,4,6,2,5,2022,121,1,17,14,07familylife@gmail.com,Walmart Virginia Beach
2022-05-01 05:00:00,5,6,2,5,2022,121,1,17,15,07familylife@gmail.com,Walmart Virginia Beach
2022-05-01 06:00:00,6,6,2,5,2022,121,1,17,0,07familylife@gmail.com,Walmart Virginia Beach
2022-05-01 07:00:00,7,6,2,5,2022,121,1,17,0,07familylife@gmail.com,Walmart Virginia Beach
2022-05-01 08:00:00,8,6,2,5,2022,121,1,17,0,07familylife@gmail.com,Walmart Virginia Beach


In [131]:
virg_dff['starthour'] = virg_dff.index
virg_dff["starthour"] = virg_dff["starthour"] + pd.Timedelta(hours=3) #converting pacific time to eastern time (virginia market)
virg_dff["hour"] = virg_dff["hour"] + 3
virg_dff.head()

C:\Users\V0V01S7\AppData\Local\Temp\ipykernel_13184\2427422206.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  virg_dff['starthour'] = virg_dff.index
C:\Users\V0V01S7\AppData\Local\Temp\ipykernel_13184\2427422206.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  virg_dff["starthour"] = virg_dff["starthour"] + pd.Timedelta(hours=3) #converting pacific time to eastern time (virginia market)
C:\Users\V0V01S7\AppData\Local\Temp\ipykernel_13184\2427422206.py:3: SettingWithCopyWarning: 
A value is trying to

,hour,dayofweek,quarter,month,year,dayofyear,dayofmonth,weekofyear,minutes,ID,market,starthour
start_hour,,,,,,,,,,,,
2022-05-01 04:00:00,7,6,2,5,2022,121,1,17,14,07familylife@gmail.com,Walmart Virginia Beach,2022-05-01 07:00:00
2022-05-01 05:00:00,8,6,2,5,2022,121,1,17,15,07familylife@gmail.com,Walmart Virginia Beach,2022-05-01 08:00:00
2022-05-01 06:00:00,9,6,2,5,2022,121,1,17,0,07familylife@gmail.com,Walmart Virginia Beach,2022-05-01 09:00:00
2022-05-01 07:00:00,10,6,2,5,2022,121,1,17,0,07familylife@gmail.com,Walmart Virginia Beach,2022-05-01 10:00:00
2022-05-01 08:00:00,11,6,2,5,2022,121,1,17,0,07familylife@gmail.com,Walmart Virginia Beach,2022-05-01 11:00:00


In [132]:
virg_dff = virg_dff.set_index('starthour',drop=True)
virg_dff


,hour,dayofweek,quarter,month,year,dayofyear,dayofmonth,weekofyear,minutes,ID,market
starthour,,,,,,,,,,,
2022-05-01 07:00:00,7,6,2,5,2022,121,1,17,14,07familylife@gmail.com,Walmart Virginia Beach
2022-05-01 08:00:00,8,6,2,5,2022,121,1,17,15,07familylife@gmail.com,Walmart Virginia Beach
2022-05-01 09:00:00,9,6,2,5,2022,121,1,17,0,07familylife@gmail.com,Walmart Virginia Beach
2022-05-01 10:00:00,10,6,2,5,2022,121,1,17,0,07familylife@gmail.com,Walmart Virginia Beach
2022-05-01 11:00:00,11,6,2,5,2022,121,1,17,0,07familylife@gmail.com,Walmart Virginia Beach
...,...,...,...,...,...,...,...,...,...,...,...
2022-06-11 14:00:00,14,5,2,6,2022,162,11,23,0,lovejadelinn@gmail.com,Walmart Virginia Beach
2022-06-11 15:00:00,15,5,2,6,2022,162,11,23,0,lovejadelinn@gmail.com,Walmart Virginia Beach
2022-06-11 16:00:00,16,5,2,6,2022,162,11,23,0,lovejadelinn@gmail.com,Walmart Virginia Beach


In [134]:
orl_dff = dff[dff['market'] == 'Walmart Orlando']
orl_dff['starthour'] = orl_dff.index
orl_dff["starthour"] = orl_dff["starthour"] + pd.Timedelta(hours=3) #converting pacific time to eastern time (virginia market)
orl_dff["hour"] = orl_dff["hour"] + 3
orl_dff = orl_dff.set_index('starthour',drop=True)
orl_dff.head()

C:\Users\V0V01S7\AppData\Local\Temp\ipykernel_13184\2408201564.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  orl_dff['starthour'] = orl_dff.index
C:\Users\V0V01S7\AppData\Local\Temp\ipykernel_13184\2408201564.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  orl_dff["starthour"] = orl_dff["starthour"] + pd.Timedelta(hours=3) #converting pacific time to eastern time (virginia market)
C:\Users\V0V01S7\AppData\Local\Temp\ipykernel_13184\2408201564.py:4: SettingWithCopyWarning: 
A value is trying to be 

,hour,dayofweek,quarter,month,year,dayofyear,dayofmonth,weekofyear,minutes,ID,market
starthour,,,,,,,,,,,
2022-05-01 07:00:00,7,6,2,5,2022,121,1,17,1,hugoalejandrocr3@gmail.com,Walmart Orlando
2022-05-01 08:00:00,8,6,2,5,2022,121,1,17,13,hugoalejandrocr3@gmail.com,Walmart Orlando
2022-05-01 09:00:00,9,6,2,5,2022,121,1,17,17,hugoalejandrocr3@gmail.com,Walmart Orlando
2022-05-01 10:00:00,10,6,2,5,2022,121,1,17,10,hugoalejandrocr3@gmail.com,Walmart Orlando
2022-05-01 11:00:00,11,6,2,5,2022,121,1,17,2,hugoalejandrocr3@gmail.com,Walmart Orlando


In [135]:
minn_dff = dff[dff['market'] == 'Walmart South Minneapolis']
minn_dff['starthour'] = minn_dff.index
minn_dff["starthour"] = minn_dff["starthour"] + pd.Timedelta(hours=2) #converting pacific time to central time (Minn market)
minn_dff["hour"] = minn_dff["hour"] + 2
minn_dff = minn_dff.set_index('starthour',drop=True)
minn_dff.head()

C:\Users\V0V01S7\AppData\Local\Temp\ipykernel_13184\625595973.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  minn_dff['starthour'] = minn_dff.index
C:\Users\V0V01S7\AppData\Local\Temp\ipykernel_13184\625595973.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  minn_dff["starthour"] = minn_dff["starthour"] + pd.Timedelta(hours=2) #converting pacific time to central time (Minn market)
C:\Users\V0V01S7\AppData\Local\Temp\ipykernel_13184\625595973.py:4: SettingWithCopyWarning: 
A value is trying to be set

,hour,dayofweek,quarter,month,year,dayofyear,dayofmonth,weekofyear,minutes,ID,market
starthour,,,,,,,,,,,
2022-05-01 08:00:00,8,6,2,5,2022,121,1,17,35,crv10174@gmail.com,Walmart South Minneapolis
2022-05-01 09:00:00,9,6,2,5,2022,121,1,17,48,crv10174@gmail.com,Walmart South Minneapolis
2022-05-01 10:00:00,10,6,2,5,2022,121,1,17,5,crv10174@gmail.com,Walmart South Minneapolis
2022-05-01 11:00:00,11,6,2,5,2022,121,1,17,0,crv10174@gmail.com,Walmart South Minneapolis
2022-05-01 12:00:00,12,6,2,5,2022,121,1,17,0,crv10174@gmail.com,Walmart South Minneapolis


In [136]:
tul_dff = dff[dff['market'] == 'Walmart Tulsa']
tul_dff['starthour'] = tul_dff.index
tul_dff["starthour"] = tul_dff["starthour"] + pd.Timedelta(hours=2) #converting pacific time to central time (Minn market)
tul_dff["hour"] = tul_dff["hour"] + 2
tul_dff = tul_dff.set_index('starthour',drop=True)
tul_dff.head()

C:\Users\V0V01S7\AppData\Local\Temp\ipykernel_13184\3602987289.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tul_dff['starthour'] = tul_dff.index
C:\Users\V0V01S7\AppData\Local\Temp\ipykernel_13184\3602987289.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tul_dff["starthour"] = tul_dff["starthour"] + pd.Timedelta(hours=2) #converting pacific time to central time (Minn market)
C:\Users\V0V01S7\AppData\Local\Temp\ipykernel_13184\3602987289.py:4: SettingWithCopyWarning: 
A value is trying to be set 

,hour,dayofweek,quarter,month,year,dayofyear,dayofmonth,weekofyear,minutes,ID,market
starthour,,,,,,,,,,,
2022-05-02 13:00:00,13,0,2,5,2022,122,2,18,16,celc42@gmail.com,Walmart Tulsa
2022-05-02 14:00:00,14,0,2,5,2022,122,2,18,16,celc42@gmail.com,Walmart Tulsa
2022-05-02 15:00:00,15,0,2,5,2022,122,2,18,16,celc42@gmail.com,Walmart Tulsa
2022-05-02 16:00:00,16,0,2,5,2022,122,2,18,16,celc42@gmail.com,Walmart Tulsa
2022-05-02 17:00:00,17,0,2,5,2022,122,2,18,0,celc42@gmail.com,Walmart Tulsa


In [138]:
sjs_dff = dff[dff['market'] == 'Walmart San Jose South']
sjn_dff = dff[dff['market'] == 'Walmart San Jose North']

In [140]:
sjs_dff['starthour'] = sjs_dff.index
sjs_dff = sjs_dff.set_index('starthour',drop = True)

sjn_dff['starthour'] = sjn_dff.index
sjn_dff = sjn_dff.set_index('starthour',drop = True)

C:\Users\V0V01S7\AppData\Local\Temp\ipykernel_13184\291565191.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sjs_dff['starthour'] = sjs_dff.index
C:\Users\V0V01S7\AppData\Local\Temp\ipykernel_13184\291565191.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sjn_dff['starthour'] = sjn_dff.index


In [148]:
dff = pd.concat([virg_dff,minn_dff,tul_dff,orl_dff,sjs_dff,sjn_dff], ignore_index = False)
dff

,hour,dayofweek,quarter,month,year,dayofyear,dayofmonth,weekofyear,minutes,ID,market
starthour,,,,,,,,,,,
2022-05-01 07:00:00,7,6,2,5,2022,121,1,17,14,07familylife@gmail.com,Walmart Virginia Beach
2022-05-01 08:00:00,8,6,2,5,2022,121,1,17,15,07familylife@gmail.com,Walmart Virginia Beach
2022-05-01 09:00:00,9,6,2,5,2022,121,1,17,0,07familylife@gmail.com,Walmart Virginia Beach
2022-05-01 10:00:00,10,6,2,5,2022,121,1,17,0,07familylife@gmail.com,Walmart Virginia Beach
2022-05-01 11:00:00,11,6,2,5,2022,121,1,17,0,07familylife@gmail.com,Walmart Virginia Beach
...,...,...,...,...,...,...,...,...,...,...,...
2022-06-11 06:00:00,6,5,2,6,2022,162,11,23,0,soemin7432@gmail.com,Walmart San Jose North
2022-06-11 07:00:00,7,5,2,6,2022,162,11,23,0,soemin7432@gmail.com,Walmart San Jose North
2022-06-11 08:00:00,8,5,2,6,2022,162,11,23,14,soemin7432@gmail.com,Walmart San Jose North


In [149]:
dff = dff.drop(['quarter','dayofyear','weekofyear'],axis=1)
dff.head()

,hour,dayofweek,month,year,dayofmonth,minutes,ID,market
starthour,,,,,,,,
2022-05-01 07:00:00,7,6,5,2022,1,14,07familylife@gmail.com,Walmart Virginia Beach
2022-05-01 08:00:00,8,6,5,2022,1,15,07familylife@gmail.com,Walmart Virginia Beach
2022-05-01 09:00:00,9,6,5,2022,1,0,07familylife@gmail.com,Walmart Virginia Beach
2022-05-01 10:00:00,10,6,5,2022,1,0,07familylife@gmail.com,Walmart Virginia Beach
2022-05-01 11:00:00,11,6,5,2022,1,0,07familylife@gmail.com,Walmart Virginia Beach


In [150]:
def slot_calc(hour):
    
    if hour in range(6,12):
        return "morning"
    elif hour in range(12,17):
        return "afternoon"
    elif hour in range(17,22):
        return "evening"

In [151]:
dff["slot"] = dff["hour"].apply(slot_calc)

In [154]:
dff.head(40)

,hour,dayofweek,month,year,dayofmonth,minutes,ID,market,slot
starthour,,,,,,,,,
2022-05-01 07:00:00,7,6,5,2022,1,14,07familylife@gmail.com,Walmart Virginia Beach,morning
2022-05-01 08:00:00,8,6,5,2022,1,15,07familylife@gmail.com,Walmart Virginia Beach,morning
2022-05-01 09:00:00,9,6,5,2022,1,0,07familylife@gmail.com,Walmart Virginia Beach,morning
2022-05-01 10:00:00,10,6,5,2022,1,0,07familylife@gmail.com,Walmart Virginia Beach,morning
2022-05-01 11:00:00,11,6,5,2022,1,0,07familylife@gmail.com,Walmart Virginia Beach,morning
2022-05-01 12:00:00,12,6,5,2022,1,0,07familylife@gmail.com,Walmart Virginia Beach,afternoon
2022-05-01 13:00:00,13,6,5,2022,1,0,07familylife@gmail.com,Walmart Virginia Beach,afternoon
2022-05-01 14:00:00,14,6,5,2022,1,0,07familylife@gmail.com,Walmart Virginia Beach,afternoon
2022-05-01 15:00:00,15,6,5,2022,1,0,07familylife@gmail.com,Walmart Virginia Beach,afternoon


In [155]:
dff.to_csv('Desktop/6markets_hourly.csv')

In [40]:
code = """select A.DRVR_USER_ID, A.MARKET_NM
from `wmt-edw-prod.WW_GEC_VM.DRVR` A inner join `wmt-edw-prod.WW_GEC_VM.DRVR_APPLN_ACTV` B on A.DRVR_USER_ID = B.DRVR_USER_ID
where A.MARKET_NM in ('Walmart San Jose North','Walmart San Jose South','Walmart Orlando','Walmart Virginia Beach', 'Walmart Tulsa','Walmart South Minneapolis') AND B.ACTV_NM = "ONLINE"
"""
df2 = query_bq(code)
df2.head()

,DRVR_USER_ID,MARKET_NM
0,julieannpruitt@icloud.com,Walmart Tulsa
1,julieannpruitt@icloud.com,Walmart Tulsa
2,naingnainghtun088@gmail.com,Walmart Tulsa
3,banksias@aol.com,Walmart Tulsa
4,crosbyedward77@gmail.com,Walmart Tulsa


In [45]:
df2 = df2.drop_duplicates()
df2 = df2.set_index('DRVR_USER_ID',drop=True)

In [53]:
df2 = df2.reset_index(drop=False)

In [58]:
drvr_dict = dict(zip(df2.DRVR_USER_ID, df2.MARKET_NM))

In [59]:
drvr_dict

{'julieannpruitt@icloud.com': 'Walmart Tulsa',
 'naingnainghtun088@gmail.com': 'Walmart Tulsa',
 'banksias@aol.com': 'Walmart Tulsa',
 'crosbyedward77@gmail.com': 'Walmart Tulsa',
 'saschoco@aol.com': 'Walmart Tulsa',
 'chris@otorisabyss.com': 'Walmart Tulsa',
 'mz.trouble75@gmail.com': 'Walmart Tulsa',
 'billywilliams23.bw@gmail.com': 'Walmart Tulsa',
 'fernestolopez@gmail.com': 'Walmart Tulsa',
 'moneytraininc1121@gmail.com': 'Walmart Tulsa',
 'edi.chelsea@hotmail.com': 'Walmart Tulsa',
 'edonkey11@att.net': 'Walmart Tulsa',
 'riosrfrf@gmail.com': 'Walmart Tulsa',
 'salvador.ignacio.salvatierra@gmail.com': 'Walmart Tulsa',
 'investmentladi99@gmail.com': 'Walmart Tulsa',
 'jhonnymora1312@gmail.com': 'Walmart Tulsa',
 'shopandmedia@gmail.com': 'Walmart Tulsa',
 'yuriangel27@hotmail.com': 'Walmart Tulsa',
 'bryan.banks1989@gmail.com': 'Walmart Tulsa',
 'sunneec3@yahoo.com': 'Walmart Tulsa',
 'andresmmatheusq74@gmail.com': 'Walmart Tulsa',
 'asapmoss6@gmail.com': 'Walmart Tulsa',
 'ale86

In [60]:
dff['market'] = dff['ID'].map(drvr_dict)

In [62]:
dff.to_csv('Desktop/6markets_hourly.csv')

In [65]:
hourprobs = {}
for i in dfs_set.keys():
    temp = dfs_set[i]
    hourprobs[i] = temp[temp['minutes']>0].groupby('hour').size().divide(temp.groupby('hour').size())

In [71]:
hpf = pd.DataFrame(hourprobs).fillna(0).transpose()

In [72]:
hpf = hpf.reset_index(drop=False)
hpf

hour,index,0,1,2,3,4,5,6,7,8,...,14,15,16,17,18,19,20,21,22,23
0,07familylife@gmail.com,0.0,0.0,0.000000,0.147727,0.516854,0.471910,0.494382,0.471910,0.494382,...,0.000000,0.034091,0.011364,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
1,vallejoa32@yahoo.com,0.0,0.0,0.000000,0.000000,0.279070,0.586207,0.620690,0.597701,0.643678,...,0.279070,0.069767,0.058140,0.034884,0.000000,0.0,0.0,0.0,0.0,0.0
2,hugoalejandrocr3@gmail.com,0.0,0.0,0.000000,0.125000,0.483146,0.606742,0.662921,0.674157,0.651685,...,0.693182,0.602273,0.602273,0.409091,0.034091,0.0,0.0,0.0,0.0,0.0
3,kevin_miami_2015@outlook.com,0.0,0.0,0.023256,0.325581,0.569767,0.569767,0.581395,0.616279,0.639535,...,0.593023,0.593023,0.488372,0.255814,0.116279,0.0,0.0,0.0,0.0,0.0
4,contreras_correia@hotmail.com,0.0,0.0,0.000000,0.011364,0.707865,0.719101,0.797753,0.808989,0.797753,...,0.852273,0.818182,0.738636,0.670455,0.397727,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2884,ajacolby5@gmail.com,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,...,0.000000,1.000000,0.000000,0.000000,0.000000,0.0,1.0,0.0,0.0,0.0
2885,adeolasan15@gmail.com,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.037037,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
2886,carlose_cuellob@hotmail.com,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.066667,0.066667,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
2887,gabyfany31@gmail.com,0.0,0.0,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0


In [75]:
pd.set_option('display.max_columns', None)

In [76]:
hpf['market'] = hpf['index'].map(drvr_dict)
hpf

hour,index,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,market
0,07familylife@gmail.com,0.0,0.0,0.000000,0.147727,0.516854,0.471910,0.494382,0.471910,0.494382,0.488636,0.363636,0.284091,0.136364,0.068182,0.000000,0.034091,0.011364,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,Walmart Virginia Beach
1,vallejoa32@yahoo.com,0.0,0.0,0.000000,0.000000,0.279070,0.586207,0.620690,0.597701,0.643678,0.616279,0.569767,0.523256,0.511628,0.430233,0.279070,0.069767,0.058140,0.034884,0.000000,0.0,0.0,0.0,0.0,0.0,Walmart Virginia Beach
2,hugoalejandrocr3@gmail.com,0.0,0.0,0.000000,0.125000,0.483146,0.606742,0.662921,0.674157,0.651685,0.707865,0.741573,0.693182,0.647727,0.715909,0.693182,0.602273,0.602273,0.409091,0.034091,0.0,0.0,0.0,0.0,0.0,Walmart Orlando
3,kevin_miami_2015@outlook.com,0.0,0.0,0.023256,0.325581,0.569767,0.569767,0.581395,0.616279,0.639535,0.627907,0.627907,0.616279,0.563218,0.581395,0.593023,0.593023,0.488372,0.255814,0.116279,0.0,0.0,0.0,0.0,0.0,Walmart Orlando
4,contreras_correia@hotmail.com,0.0,0.0,0.000000,0.011364,0.707865,0.719101,0.797753,0.808989,0.797753,0.829545,0.852273,0.852273,0.875000,0.852273,0.852273,0.818182,0.738636,0.670455,0.397727,0.0,0.0,0.0,0.0,0.0,Walmart Orlando
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2884,ajacolby5@gmail.com,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.0,1.0,0.0,0.0,0.0,Walmart Tulsa
2885,adeolasan15@gmail.com,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.037037,0.037037,0.000000,0.037037,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,Walmart South Minneapolis
2886,carlose_cuellob@hotmail.com,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.066667,0.066667,0.066667,0.066667,0.066667,0.066667,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,Walmart Orlando
2887,gabyfany31@gmail.com,0.0,0.0,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,Walmart Orlando


In [74]:
hpf.to_csv('Desktop/hourprobs.csv')